## Aula 5.2.Coleta de dados fundamentalistas

### Exemplo 1 - Coletando a lista de companhias com registro na CVM

Acessar o site https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx

In [1]:
import requests_html
import re
import pandas as pd

In [2]:
import os
import sys
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
CLASSES_DIR = os.getenv("CLASSES_DIR")

sys.path.append(CLASSES_DIR)

In [3]:
from listar_empresas_registradas_cvm import EmpresasRegistradasCVM

reg_emp = EmpresasRegistradasCVM()
reg_emp.empresas.head()

,key,value,status
0,018554,"PAN FINANCEIRA S.A. - CRÉDITO, FINANCIAMENTO ...",REGISTRO ATIVO
1,252224,2W ECOBANK S.A.,NaN
2,025224,2W ECOBANK S.A.,REGISTRO ATIVO
3,917581,2W ENERGIA S.A.,REGISTRO ATIVO
4,021954,3A COMPANHIA SECURITIZADORA,REGISTRO CANCELADO


In [7]:
from listar_documentos_cvm import ListarDocumentosCVM

Docs_CVM = ListarDocumentosCVM(['021610','025461'], '01/01/2021', '01/01/2023')

Docs_CVM.obter_documentos()
doc_cvm_res = Docs_CVM.cvt_response_to_df()

doc_cvm_res.head()

,Cod_CVM,Sequencia,Versao,Protocolo,Tipo
0,025461,121762,1,025461ITR300920220100121762-72,ITR
1,021610,121714,1,021610ITR300920220100121714-75,ITR
2,025461,119796,1,025461ITR300620220100119796-76,ITR
3,021610,119633,1,021610ITR300620220100119633-78,ITR
4,021610,114979,1,021610ITR310320220100114979-74,ITR


In [9]:
from database.sqlite_db import DatabaseConnectionFactory

conn = DatabaseConnectionFactory.CreateDbConnection()
c = conn.cursor()

# c.execute("delete from documentos_cvm")
# conn.commit()

for row, doc in doc_cvm_res.iterrows():
    sql = f"REPLACE INTO documentos_cvm VALUES ('{doc['Cod_CVM']}', {doc['Sequencia']}, {doc['Versao']}, '{doc['Protocolo']}', '{doc['Tipo']}')"
    c.execute(sql)

conn.commit()

result = c.execute("select * from documentos_cvm").fetchall()
conn.close()

result

[('021610', 103505, 1, '021610ITR310320210100103505-72', 'ITR'),
 ('025461', 103710, 1, '025461ITR310320210100103710-75', 'ITR'),
 ('025461', 103735, 2, '025461ITR310320210200103735-68', 'ITR'),
 ('025461', 104287, 3, '025461ITR310320210300104287-72', 'ITR'),
 ('025461', 107406, 1, '025461ITR300620210100107406-78', 'ITR'),
 ('025461', 107427, 2, '025461ITR300620210200107427-72', 'ITR'),
 ('025461', 107438, 3, '025461ITR300620210300107438-76', 'ITR'),
 ('021610', 107516, 1, '021610ITR300620210100107516-76', 'ITR'),
 ('025461', 107530, 4, '025461ITR300620210400107530-79', 'ITR'),
 ('025461', 109616, 1, '025461ITR300920210100109616-84', 'ITR'),
 ('025461', 109702, 2, '025461ITR300920210200109702-71', 'ITR'),
 ('021610', 109787, 1, '021610ITR300920210100109787-89', 'ITR'),
 ('025461', 114673, 1, '025461ITR310320220100114673-74', 'ITR'),
 ('021610', 114979, 1, '021610ITR310320220100114979-74', 'ITR'),
 ('021610', 119633, 1, '021610ITR300620220100119633-78', 'ITR'),
 ('025461', 119796, 1, '0

### Exemplo 2 - Coleta de dados da CVM usando requests-html

No exemplo a seguir vamos primeiramente entender como o site de documentos da CVM funciona.

Vamos inicialmente inspecionar o site https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx

In [2]:
df_empresas = pd.read_csv('registros_empresas_cvm.csv', sep=';')

In [3]:
from download_doc_xml_rad_cvm import DownloadDocXmlRadCvm
from database.Documento_CVM import Documento_CVM

teste_doc = Documento_CVM(('021610', 103505, 1, '021610ITR310320210100103505-72', 'ITR'))
down_doc = DownloadDocXmlRadCvm(teste_doc)
down_doc.download_xml()
down_doc.open_xml()

In [ ]:
my_dict = down_doc.dados

In [ ]:
my_dict

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario'].keys()

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas']

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'].keys()

#### Balanço Patrimonial Ativo

In [ ]:
doc = "BalancoPatrimonialAtivo"

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]

In [ ]:
df_bpa = pd.DataFrame()
for row in my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]['Conta']:
    df_bpa = pd.concat([df_bpa, pd.DataFrame([row])])

In [ ]:
df_bpa

#### Balanço Patrimonial Passivo

In [ ]:
doc = "BalancoPatrimonialPassivo"

In [ ]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]

In [ ]:
df_bpp = pd.DataFrame()
for row in my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]['Conta']:
    df_bpp = pd.concat([df_bpp, pd.DataFrame([row])])

In [ ]:
df_bpp